In [ ]:
#%%writefile sphere_geom.py

# 11-07-19

'''
Builds spherical boundary from decahedron
'''

import numpy as np
import os

def decahedron_points(r):
    '''
    Returns vert_array and face_array for a given radii (r) in armstrong.
    Note: Do not move the center of the sphere! This may cause troubles on smoothing function.
    '''
    
    # from http://www.sacred-geometry.es/?q=es/content/phi-en-los-s%C3%B3lidos-sagrados
    
    rho = (1. + np.sqrt(5.)) * 0.5
    
    if r==0:
        return None
    
    scale = (r) /np.sqrt(1.+rho**2.)
    
    vert_array = np.zeros((12,3))
    
    vert_array[0]  = np.array((   0. , -rho  ,   1. ))
    vert_array[1]  = np.array((   0. ,  rho  ,   1. ))
    vert_array[2]  = np.array((   0. ,  rho  ,  -1. ))
    vert_array[3]  = np.array((   0. , -rho  ,  -1. ))
    vert_array[4]  = np.array((   1. ,  0.   ,  rho ))
    vert_array[5]  = np.array((  -1. ,  0.   ,  rho ))
    vert_array[6]  = np.array((  -1. ,  0.   , -rho ))
    vert_array[7]  = np.array((   1. ,  0.   , -rho ))
    vert_array[8]  = np.array((  rho ,  1.   ,   0. ))
    vert_array[9]  = np.array(( -rho ,  1.   ,   0. ))
    vert_array[10] = np.array(( -rho , -1.   ,   0. ))
    vert_array[11] = np.array((  rho , -1.   ,   0. ))
    
    face_array = np.zeros((20,3))
    
    face_array[0]  = np.array(( 11 ,  7 ,  8 )) 
    face_array[1]  = np.array((  8 ,  7 ,  2 )) 
    face_array[2]  = np.array((  8 ,  2 ,  1 )) 
    face_array[3]  = np.array((  1 ,  2 ,  9 )) 
    face_array[4]  = np.array((  1 ,  9 ,  5 )) 
    face_array[5]  = np.array((  1 ,  5 ,  4 )) 
    face_array[6]  = np.array((  5 ,  0 ,  4 )) 
    face_array[7]  = np.array((  4 ,  0 , 11 )) 
    face_array[8]  = np.array((  7 ,  3 ,  6 )) 
    face_array[9]  = np.array(( 11 ,  8 ,  4 )) 
    face_array[10] = np.array((  8 ,  1 ,  4 ))
    face_array[11] = np.array((  6 , 10 ,  9 ))
    face_array[12] = np.array((  2 ,  6 ,  9 ))
    face_array[13] = np.array(( 10 ,  3 ,  0 ))
    face_array[14] = np.array(( 10 ,  0 ,  5 ))
    face_array[15] = np.array((  5 ,  9 , 10 ))
    face_array[16] = np.array((  6 ,  3 , 10 ))
    face_array[17] = np.array((  3 , 11 ,  0 ))
    face_array[18] = np.array((  7 , 11 ,  3 ))
    face_array[19] = np.array((  7 ,  6 ,  2 ))
        
    return vert_array*scale , face_array.astype(int)

def smothing_func( v_i , r ):
    '''
    Softens the boundary to achieve spherical behaivour.
    v_i : vertex to move to the boundary
    r   : sphere radii
    '''
    t = r /np.linalg.norm(v_i)
    return v_i * t

def pqr_assembly( x_q , q , mol_name ):
    '''
    Writtes {mol_name}.pqr file for a given position (x_q) and charges (q).
    '''
    
    pqr_text = open( os.path.join( 'Molecule' , mol_name , mol_name +'.pqr' ) , 'w+' )
    
    # .pqr format is:
    # recordName serial atomName residueName chainID residueNumber X Y Z charge radius
    
    if np.shape(x_q) == (3,):
        position = ' '.join( x_q.astype(str) )
        
        radii = 1. # Is implicitly defined on sphere radii
        
        text = 'ION      {0} None {1}   1      {2}  {3}  {4:5f} \n'.format(
        1 , mol_name , position , q , radii)
        
        pqr_text.write(text)
        pqr_text.close()
    
    else:
        count = 0
        for x in x_q:
            position = ' '.join( x.astype(str) )

            radii = 1. # Is implicitly defined on sphere radii

            text = 'ION      {0} None {1}   1      {2}  {3}  {4:5f} \n'.format(
            count+1 , mol_name , position , q[count] , radii)

            pqr_text.write(text)

            count+=1

        pqr_text.close()
    return None

def suffix_names(N_s):
    instances = np.empty( (N_s+1 , )).astype(str)
    for i in range(N_s +1 ):
        instances[i] = '-s{0:d}'.format(i)
    return instances

def moving_files(name , percentaje , s):


    path      = os.path.join( 'Molecule' , name)
    dest_path = os.path.join( path , str( int(percentaje*100)) , str(s) )
    os.system('mv {0}*.vtk {0}*.face {0}*.vert {0}*.msh {0}*.off {1}'.format(path+'/' , dest_path) )
    
    return

In [23]:
import numpy as np
from math import pi
import os
import time

from constants import *
#from constants import mesh_info

#from Grid_Maker_R2  import *
#from Mesh_Ref_V2    import *
from quadrature     import *
#from Potential_Solver import *
from sphere_geom    import *
#from bem_parameters import *

from analytical     import *

def radii( i , R ):
    return (R - (0.95)**i )

Radius = 1.0
N = (22 , 25 , 30 )
a = 1.0

text_name = 'Failed_Test'

testing_Radii = True
testing_theta = False

if testing_Radii: text_name = 'Resultados_Q_k'
if testing_theta: text_name = 'Resultados_Q_k_theta'

Results = open( text_name + '.txt' , 'w+')
if testing_Radii: Results.write('Radius   22     25     30   \n')
if testing_theta: Results.write('theta    22     25     30   \n')
Results.close()

max_points = 40


for i in range(max_points):
    
    if testing_Radii:   line = str(radii( i , Radius ))
    if testing_theta:   line =  str( np.pi*(2.*i/max_points ) ) + ' ' + str( 180.*(2.*i/max_points ) )
    
    for n in N:
        if testing_Radii:
            x_q =  np.array( [[  1.E-12 ,  1.E-12 ,  radii( i , Radius ) ]])
                             
            q = np.array( [1. ])
                             
            f_ex = solution(q, x_q, ep_m, ep_s, Radius, k , a, n)

            line = line + ' ' + str(f_ex)
                             
        if testing_theta:
            x_q = np.array( [[  1.E-12 ,  1.E-12 ,  0.62 ],
                             [  1.E-12 ,  0.62*np.cos(np.pi*(1.5+(i+1)/max_points) ) ,
                                                      0.62*np.sin(np.pi*(1.5+(i+1)/max_points) )  ] ,
                             [  1.E-12 ,  0.62*np.cos(np.pi*(1.5-(i+1)/max_points)) ,
                                                      0.62*np.sin(np.pi*(1.5-(i+1)/max_points) )  ]
                            ])
            q = np.array( [1. , 1. , -1.]) 

            f_ex = solution(q, x_q, ep_m, ep_s, Radius , k , a, n)

            line = line + ' ' + str(f_ex)
        
    
    Results = open(text_name +  '.txt' , 'a+')
    Results.write(line + '\n')
    Results.close()
    
Results = open(text_name + '.txt' , 'a+')
if testing_Radii:
    Results.write('Description: Solvation energy moving the charge closer to the boundary.')
if testing_theta:
    Results.write('Description: Solvation energy moving the charges closer. 3 charges tested.')                       
Results.close()
    


In [25]:
# Read text file from previus doc

text = open('Resultados_Q_k.txt','r').read().split('\n')

for line in text[1:-1]:
    radius , Solv_20 , _ , Solv_30 =  map(float , line.split(' ') )
    #angle_rad , angle_deg , Solv_20 , Solv_25 , Solv_30 = map(float , line.split(' ') )
    print('Solvation relative Residual 20/30: {0:.4e}  @{1:.7f}'.format( (Solv_20 - Solv_30)/Solv_30 ,
                                                                        #angle_deg) )
                                                                        radius ) )

Solvation relative Residual 20/30: -0.0000e+00  @0.0000000
Solvation relative Residual 20/30: -0.0000e+00  @0.0500000
Solvation relative Residual 20/30: -0.0000e+00  @0.0975000
Solvation relative Residual 20/30: -0.0000e+00  @0.1426250
Solvation relative Residual 20/30: -0.0000e+00  @0.1854938
Solvation relative Residual 20/30: -0.0000e+00  @0.2262191
Solvation relative Residual 20/30: -0.0000e+00  @0.2649081
Solvation relative Residual 20/30: -0.0000e+00  @0.3016627
Solvation relative Residual 20/30: -0.0000e+00  @0.3365796
Solvation relative Residual 20/30: -0.0000e+00  @0.3697506
Solvation relative Residual 20/30: -0.0000e+00  @0.4012631
Solvation relative Residual 20/30: -1.4669e-16  @0.4311999
Solvation relative Residual 20/30: -1.1379e-15  @0.4596399
Solvation relative Residual 20/30: -1.6530e-14  @0.4866579
Solvation relative Residual 20/30: -1.5869e-13  @0.5123250
Solvation relative Residual 20/30: -1.2298e-12  @0.5367088
Solvation relative Residual 20/30: -7.8979e-12  @0.55987

In [28]:
x_q = np.array( [[  1.E-12 ,  1.E-12 ,  0.62 ],
                             [  1.E-12 ,  0.62*np.cos(np.pi*1.5 + 5.*np.pi/180.) ,
                                                      0.62*np.sin(np.pi*1.5 + 5.*np.pi/180. ) ] ,
                             [  1.E-12 ,  0.62*np.cos(np.pi*1.5 - 5.*np.pi/180.) ,
                                                      0.62*np.sin(np.pi*1.5 - 5.*np.pi/180. )  ]
                            ])
q = np.array( [1. , 1. , -1.]) 

f_ex = solution(q, x_q, ep_m, ep_s, Radius , k , a, n)

f_ex

-65.46725659445093

In [1]:
for i in range(100):
    print(i/100)

0.0
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99


In [ ]:

import bempp.api, numpy as np
from math import pi
import os
import time

from constants import values
from constants import mesh_info

from bempp.api.operators.potential import laplace as lp
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

from Grid_Maker_R2  import *
from Mesh_Ref_V2    import *
from quadrature     import *
from Potential_Solver import *
from sphere_geom    import *
#from bem_parameters import *

from analytical     import *


#global mol_name , mesh_density , suffix , path , q , x_q , phi_space , phi_order , u_space , u_order

def main2(name , dens , input_suffix , output_suffix , percentaje , r , x_q
         , q , smooth = False , refine = True , test_plot = False):
    
    init_time = time.time()

    mesh_info.mol_name     = name
    mesh_info.mesh_density = dens
    mesh_info.suffix       = input_suffix
    mesh_info.path         = os.path.join('Molecule' , mesh_info.mol_name)
    
    print(mesh_info.suffix)

    mesh_info.q , mesh_info.x_q = q , x_q

    mesh_info.u_space , mesh_info.u_order     = 'DP' , 0
    mesh_info.phi_space , mesh_info.phi_order = 'P' , 1
    mesh_info.u_s_space , mesh_info.u_s_order = 'P' , 1


    bempp.api.set_ipython_notebook_viewer()
    bempp.api.PLOT_BACKEND = "ipython_notebook"

    grid = Grid_loader( mesh_info.mol_name , mesh_info.mesh_density , mesh_info.suffix )

    dirichl_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)
    neumann_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order) 
    dual_to_dir_s_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)

    u_s  = bempp.api.GridFunction(dirichl_space_u, fun=u_s_G )
    du_s = bempp.api.GridFunction(neumann_space_u, fun=du_s_G)

    #u_s.plot()

    u_h , du_h = harmonic_component(dirichl_space_u , neumann_space_u , dual_to_dir_s_u , u_s , du_s)
    u_r , du_r = regular_component(dirichl_space_u , neumann_space_u , dual_to_dir_s_u , du_s , du_h)

    S_trad = S_trad_calc( dirichl_space_u, neumann_space_u , u_h , du_h , u_r , du_r)

    endt_time = time.time()
    fin_time = endt_time - init_time

    dirichl_space_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order)
    neumann_space_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order) 
    dual_to_dir_s_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order)

    phi , dphi = adjoint_equation( dirichl_space_phi , neumann_space_phi , dual_to_dir_s_phi)

    aux_path = '_'+str(mesh_info.mesh_density)+ mesh_info.suffix

    face_array = text_to_list(mesh_info.mol_name , aux_path , '.face' , info_type=int  )
    vert_array = text_to_list(mesh_info.mol_name , aux_path , '.vert' , info_type=float)

    S_Cooper , S_Cooper_i = S_Cooper_calc( face_array , vert_array , phi , dphi , u_r+u_h , du_r+du_h , 25)

    dirichl_space_u_s  = bempp.api.function_space(grid,  mesh_info.u_s_space , mesh_info.u_s_order )
    neumann_space_du_s = bempp.api.function_space(grid,  mesh_info.u_s_space , mesh_info.u_s_order )

    u_s  = bempp.api.GridFunction(dirichl_space_u_s , fun=u_s_G )
    du_s = bempp.api.GridFunction(neumann_space_du_s, fun=du_s_G)      

    S_Zeb    , S_Zeb_i    = S_Zeb_calc( face_array , vert_array , phi , dphi , u_s , du_s , 25)
                                # Zeb_aproach_with_u_s_Teo( face_array , vert_array , phi , dphi , 25)

    const_space = bempp.api.function_space(grid,  "DP", 0)

    Solv_i_Cooper_Func = bempp.api.GridFunction(const_space, fun=None, coefficients=S_Cooper_i[:,0])
    Solv_i_Zeb_Func    = bempp.api.GridFunction(const_space, fun=None, coefficients=S_Zeb_i[:,0])

    dif =S_Cooper_i-S_Zeb_i

    dif_F = bempp.api.GridFunction(const_space, fun=None, coefficients=np.abs(dif[:,0] ) )

    #dif_F.plot()    

    bempp.api.export('Molecule/' + name +'/' + name + '_{0}{1}.vtk'.format( dens,input_suffix )
                         , grid_function = dif_F , data_type = 'element')

    #if False:
    #    status = value_assignor_starter(face_array , np.abs(dif[:,0]) , percentaje) #final_status(face_array , dif[:,0] , 2 )
    #    const_space = bempp.api.function_space(grid,  "DP", 0)
    #    Status    = bempp.api.GridFunction(const_space, fun=None, coefficients=status)
    #    Status.plot()

    if refine:

        new_face_array , new_vert_array = mesh_refiner(face_array , vert_array , np.abs(dif[:,0]) , percentaje )

        if smooth:

            aux_vert_array = np.zeros(( len(new_vert_array),3 ))

            c=0
            for vert in new_vert_array:
                aux_vert_array[c] = smothing_func(vert , r) 
                c+=1

            #print(aux_vert_array)
            
        elif not smooth:
        
            aux_vert_array = new_vert_array.copy()

    vert_and_face_arrays_to_text_and_mesh( name , aux_vert_array ,
                                                new_face_array.astype(int)[:] , output_suffix, dens , Self_build=True)

    grid = Grid_loader( name , dens , output_suffix )
    #print('New mesh:')
    if test_plot:
        
        grid.plot()

    N_elements = len(face_array)
        
    return S_trad , S_Cooper , S_Zeb , N_elements , fin_time

x_q = np.array( [[  1.E-12 ,  1.E-12 ,  0.5 ]])
                         #[  1.E-12 ,  1.E-12 ,  1.E-12]] )
q = np.array( [1. ]) #, 1. ] )

r = 1.
name = 'sphere_cent'
dens = 0

dec_v_array , dec_f_array = decahedron_points(r)

vert_and_face_arrays_to_text_and_mesh( name, dec_v_array , dec_f_array+1 , '-s0' 
                                                  , dens=0 , Self_build=True)

main2(name , 0 , '-s0' , '-s1' , 2.5 , r , x_q, q , smooth = True )
main2(name , 0 , '-s1' , '-s2' , 0.7 , r , x_q, q , smooth = True )
main2(name , 0 , '-s2' , '-s3' , 0.7 , r , x_q, q , smooth = True )
main2(name , 0 , '-s3' , '-s4' , 0.7 , r , x_q, q , smooth = True )
main2(name , 0 , '-s4' , '-s5' , 0.7 , r , x_q, q , smooth = True , test_plot = True)

In [ ]:
#%%writefile sphere_run_51_it_10p.py

# August 13th 2019
# This rutine uses the three term splitting formulation

import bempp.api, numpy as np
from math import pi
import os
import time

from constants import values
from constants import mesh_info

from bempp.api.operators.potential import laplace as lp
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

from Grid_Maker_R2  import *
from Mesh_Ref_V2    import *
from quadrature     import *
from Potential_Solver import *
from sphere_geom    import *
#from bem_parameters import *
from File_converter_python2 import *

from analytical     import *


#global mol_name , mesh_density , suffix , path , q , x_q , phi_space , phi_order , u_space , u_order

def main(name , dens , input_suffix , output_suffix , percentaje , r , x_q
         , q , smooth = False , refine = True ):
    
    init_time = time.time()

    mesh_info.mol_name     = name
    mesh_info.mesh_density = dens
    mesh_info.suffix       = input_suffix
    mesh_info.path         = os.path.join('Molecule' , mesh_info.mol_name)
    
    print('{1:.0f} {0} '.format(mesh_info.suffix , percentaje * 100.) )

    mesh_info.q , mesh_info.x_q = q , x_q

    mesh_info.u_space , mesh_info.u_order     = 'DP' , 0
    mesh_info.phi_space , mesh_info.phi_order = 'P' , 1
    mesh_info.u_s_space , mesh_info.u_s_order = 'P' , 1


    bempp.api.set_ipython_notebook_viewer()
    bempp.api.PLOT_BACKEND = "ipython_notebook"

    grid = Grid_loader( mesh_info.mol_name , mesh_info.mesh_density , mesh_info.suffix )

    dirichl_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)
    neumann_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order) 
    dual_to_dir_s_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)

    #u_s  = bempp.api.GridFunction(dirichl_space_u, fun=u_s_G )
    #du_s = bempp.api.GridFunction(neumann_space_u, fun=du_s_G)

    #u_s.plot()

    U, dU , U_R , dU_R = U_and_U_Reac(dirichl_space_u , neumann_space_u , dual_to_dir_s_u )
    
    S_trad = S_trad_calc_R( dirichl_space_u, neumann_space_u , U , dU )
    
    endt_time = time.time()
    fin_time = endt_time - init_time

    dirichl_space_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order)
    neumann_space_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order) 
    dual_to_dir_s_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order)

    phi , dphi = adjoint_equation( dirichl_space_phi , neumann_space_phi , dual_to_dir_s_phi)

    aux_path = '_'+str(mesh_info.mesh_density)+ mesh_info.suffix

    face_array = text_to_list(mesh_info.mol_name , aux_path , '.face' , info_type=int  )
    vert_array = text_to_list(mesh_info.mol_name , aux_path , '.vert' , info_type=float)

    S_Cooper , S_Cooper_i = S_Cooper_calc( face_array , vert_array , phi , dphi , U_R , dU_R , 25)

    dirichl_space_u_s  = bempp.api.function_space(grid,  mesh_info.u_s_space , mesh_info.u_s_order )
    neumann_space_du_s = bempp.api.function_space(grid,  mesh_info.u_s_space , mesh_info.u_s_order )

    u_s  = bempp.api.GridFunction(dirichl_space_u_s , fun=u_s_G )
    du_s = bempp.api.GridFunction(neumann_space_du_s, fun=du_s_G)      

    S_Zeb    , S_Zeb_i    = S_Zeb_calc( face_array , vert_array , phi , dphi , u_s , du_s , 25)
                                # Zeb_aproach_with_u_s_Teo( face_array , vert_array , phi , dphi , 25)

    const_space = bempp.api.function_space(grid,  "DP", 0)

    Solv_i_Cooper_Func = bempp.api.GridFunction(const_space, fun=None, coefficients=S_Cooper_i[:,0])
    Solv_i_Zeb_Func    = bempp.api.GridFunction(const_space, fun=None, coefficients=S_Zeb_i[:,0])

    dif =S_Cooper_i-S_Zeb_i

    dif_F = bempp.api.GridFunction(const_space, fun=None, coefficients=np.abs(dif[:,0] ) )
    #dif_F.plot()    

    bempp.api.export('Molecule/' + name +'/' + name + '_{0}{1}.vtk'.format( dens,input_suffix )
                         , grid_function = dif_F , data_type = 'element')

    #if False:
    #    status = value_assignor_starter(face_array , np.abs(dif[:,0]) , percentaje) #final_status(face_array , dif[:,0] , 2 )
    #    const_space = bempp.api.function_space(grid,  "DP", 0)
    #    Status    = bempp.api.GridFunction(const_space, fun=None, coefficients=status)
    #    Status.plot()

    if refine:

        new_face_array , new_vert_array = mesh_refiner(face_array , vert_array , np.abs(dif[:,0]) , percentaje )

        if smooth:

            aux_vert_array = np.zeros(( len(new_vert_array),3 ))

            c=0
            for vert in new_vert_array:
                aux_vert_array[c] = smothing_func(vert , r) 
                c+=1

            #print(aux_vert_array)
            
        elif not smooth:
        
            aux_vert_array = new_vert_array.copy()
            

    
    # Run over ImproveSurfMesh  --> FETK Toolkit
    # Place this program path in constants.py  !
    
    new_face_array , aux_vert_array = Improve_Mesh(new_face_array , aux_vert_array , mesh_info.path , 
                                                  mesh_info.mol_name+ '_' + str(dens) + output_suffix )
    
    
    vert_and_face_arrays_to_text_and_mesh( name , aux_vert_array ,
                                                new_face_array.astype(int)[:] , output_suffix, dens , Self_build=True)

    grid = Grid_loader( name , dens , output_suffix )
    #print('New mesh:')
    #grid.plot()

    N_elements = len(face_array)
        
    return S_trad , S_Cooper , S_Zeb , N_elements , fin_time


def rutine(name , N_it, percentajes , r = 1.):
    
    #name = 'sphere_excent'
    
    if name == 'sphere_cent':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,  1.E-12 ]]  )
        q = np.array( [1.] )
    
    if name == 'sphere_excent':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,   r/2. ]]  )
        q = np.array( [1.] )
    
    if name == 'sphere_dip_C5':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,   1.E-12 ] ,
                         [  1.E-12 ,  1.E-12 ,   0.5    ] ])
        q = np.array( [1. , 1. ] )
    
    if name == 'sphere_dip': 
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,    0.5 ] ,
                         [  1.E-12 ,  1.E-12 ,   -0.5    ] ])
        q = np.array( [1. , 1. ] )
        
    if name == 'sphere_dip_12':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,    0.5    ] ,
                         [  1.E-12 ,  1.E-12 ,   -0.5    ] ])
        q = np.array( [1. , 2. ] )
        
    #for percentaje in percentajes:
    if True:
        
        percentaje = 0.1
        #percentaje = 1.5
        
        #for s in (True,False):
        if True:
            
            s = True
            suffixes = suffix_names(N_it)

            #x_q = np.array( [[  1.E-12 ,  1.E-12 ,   1.E-12 ]]  ) #,
                           #  [  1.E-12 ,  1.E-12 ,  -0.5    ] ])
            #q = np.array( [1.] ) #, 2. ]) #, 1. ] )

            #r = 1.
            #name = 'sphere_cent'
            dens = 0

            path = os.path.join( 'Molecule' , name , str( int(percentaje*100)) , str(s) )            
            os.system('mkdir -p {0}'.format( path ) )

            #os.system('chmod -R a+xwr {0}/'.format('.'))

            bempp.api.set_ipython_notebook_viewer()
            bempp.api.PLOT_BACKEND = "ipython_notebook"

            # Creates Initial Decahedron
            dec_v_array , dec_f_array = decahedron_points(r)
            vert_and_face_arrays_to_text_and_mesh( name, dec_v_array , dec_f_array+1 , '-s0' 
                                                      , dens=0 , Self_build=True)
            # Refines to obtain a better aproach of a sphere
            main(name , dens , '-s0' , '-s1' , percentaje=2.5 , r=r , x_q=x_q , q=q , smooth = True)


            #grid = Grid_loader( name , 0 , '-s0' )
            #grid.plot()

            Resultados = open('Molecule/{0}/Resultados_{0}.txt'.format( name ) , 'w+')

            Resultados.write('x_q = \n')
            Resultados.write(str(x_q) )
            Resultados.write('\n q =   \n')
            Resultados.write(str(  q) )
            Resultados.write('  \n')


            Resultados.write('Percentaje & Number of elements & Strad & SAprox & SEx & smooth & time \n')
            Resultados.close()

            c=1
            for i in suffixes[1:-1]:

                line = main(name , dens , suffixes[c]  , suffixes[c+1] , percentaje , r , x_q , q , smooth = s )
                
                
                print(c , suffixes[c] , suffixes[c+1] )
                
                Resultados = open('Molecule/{0}/Resultados_{0}.txt'.format( name ) , 'a')
                Resultados.write('{4:.2f} & {3:d} & {0:.10f} & {1:.10f} & {2:.10f} & {5} & {6:.10f}\n'.format(
                            line[0] , line[1], line[2] , line[3] , percentaje, str(s) , line[4]))
                Resultados.close()

                #res = np.vstack( (res , np.array( (name , line[0] , line[1], line[2]
                #                                  , line[3] , percentaje, str(s) , line[4])) ) )
                c+=1
                
                #os.system( "python3.6 File_converter_python3.py" )

                if c==5 and percentaje == 1.5: break

            #R   = 1.

            a   = 1.
            N   = 20

            f_ex = solution(q, x_q, ep_m, ep_s, r, k , a, N)
            print('Exact Solution: {0:5f}--------------------------------------------'.format(f_ex))
            #print(solution_2(q, x_q, ep_m, ep_s, r, kappa, a, N))
            Resultados = open('Molecule/{0}/Resultados_{0}.txt'.format( name ) , 'a')
            Resultados.write('Exact solution: {0:10f}'.format(f_ex))
            Resultados.close()

            moving_files(name , percentaje , s)
            
    return None


#input_suffix = '-s0'
#output_suffix = '-s1'

#res = np.empty(  (0,8) )

N_it = 5
percentajes = ''

#name = 'sphere_cent'
#rutine(name , N_it, percentajes )

name = 'sphere_excent'
rutine(name , N_it, percentajes )

In [ ]:
#%%writefile sphere_run.py

# This rutine uses the three term splitting formulation

import bempp.api, numpy as np
from math import pi
import os
import time

from constants import values
from constants import mesh_info

from bempp.api.operators.potential import laplace as lp
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

from Grid_Maker_R2  import *
from Mesh_Ref_V2    import *
from quadrature     import *
from Potential_Solver import *
from sphere_geom    import *
#from bem_parameters import *
from File_converter_python2 import *

from analytical     import *


#global mol_name , mesh_density , suffix , path , q , x_q , phi_space , phi_order , u_space , u_order

def main(name , dens , input_suffix , output_suffix , percentaje , r , x_q
         , q , smooth = False , refine = True ):
    
    init_time = time.time()

    mesh_info.mol_name     = name
    mesh_info.mesh_density = dens
    mesh_info.suffix       = input_suffix
    mesh_info.path         = os.path.join('Molecule' , mesh_info.mol_name)
    
    print('{1:.0f} {0} '.format(mesh_info.suffix , percentaje * 100.) )

    mesh_info.q , mesh_info.x_q = q , x_q

    mesh_info.u_space , mesh_info.u_order     = 'DP' , 0
    mesh_info.phi_space , mesh_info.phi_order = 'P' , 1
    mesh_info.u_s_space , mesh_info.u_s_order = 'P' , 1


    bempp.api.set_ipython_notebook_viewer()
    bempp.api.PLOT_BACKEND = "ipython_notebook"

    grid = Grid_loader( mesh_info.mol_name , mesh_info.mesh_density , mesh_info.suffix )

    dirichl_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)
    neumann_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order) 
    dual_to_dir_s_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)

    u_s  = bempp.api.GridFunction(dirichl_space_u, fun=u_s_G )
    du_s = bempp.api.GridFunction(neumann_space_u, fun=du_s_G)

    #u_s.plot()

    u_h , du_h = harmonic_component(dirichl_space_u , neumann_space_u , dual_to_dir_s_u , u_s , du_s)
    u_r , du_r = regular_component(dirichl_space_u , neumann_space_u , dual_to_dir_s_u , du_s , du_h)

    S_trad = S_trad_calc( dirichl_space_u, neumann_space_u , u_h , du_h , u_r , du_r)

    endt_time = time.time()
    fin_time = endt_time - init_time

    dirichl_space_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order)
    neumann_space_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order) 
    dual_to_dir_s_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order)

    phi , dphi = adjoint_equation( dirichl_space_phi , neumann_space_phi , dual_to_dir_s_phi)

    aux_path = '_'+str(mesh_info.mesh_density)+ mesh_info.suffix

    face_array = text_to_list(mesh_info.mol_name , aux_path , '.face' , info_type=int  )
    vert_array = text_to_list(mesh_info.mol_name , aux_path , '.vert' , info_type=float)

    S_Cooper , S_Cooper_i = S_Cooper_calc( face_array , vert_array , phi , dphi , u_r+u_h , du_r+du_h , 25)

    dirichl_space_u_s  = bempp.api.function_space(grid,  mesh_info.u_s_space , mesh_info.u_s_order )
    neumann_space_du_s = bempp.api.function_space(grid,  mesh_info.u_s_space , mesh_info.u_s_order )

    u_s  = bempp.api.GridFunction(dirichl_space_u_s , fun=u_s_G )
    du_s = bempp.api.GridFunction(neumann_space_du_s, fun=du_s_G)      

    S_Zeb    , S_Zeb_i    = S_Zeb_calc( face_array , vert_array , phi , dphi , u_s , du_s , 25)
                                # Zeb_aproach_with_u_s_Teo( face_array , vert_array , phi , dphi , 25)

    const_space = bempp.api.function_space(grid,  "DP", 0)

    Solv_i_Cooper_Func = bempp.api.GridFunction(const_space, fun=None, coefficients=S_Cooper_i[:,0])
    Solv_i_Zeb_Func    = bempp.api.GridFunction(const_space, fun=None, coefficients=S_Zeb_i[:,0])

    dif =S_Cooper_i-S_Zeb_i

    dif_F = bempp.api.GridFunction(const_space, fun=None, coefficients=np.abs(dif[:,0] ) )
    #dif_F.plot()    

    bempp.api.export('Molecule/' + name +'/' + name + '_{0}{1}.vtk'.format( dens,input_suffix )
                         , grid_function = dif_F , data_type = 'element')

    #if False:
    #    status = value_assignor_starter(face_array , np.abs(dif[:,0]) , percentaje) #final_status(face_array , dif[:,0] , 2 )
    #    const_space = bempp.api.function_space(grid,  "DP", 0)
    #    Status    = bempp.api.GridFunction(const_space, fun=None, coefficients=status)
    #    Status.plot()

    if refine:

        new_face_array , new_vert_array = mesh_refiner(face_array , vert_array , np.abs(dif[:,0]) , percentaje )

        if smooth:

            aux_vert_array = np.zeros(( len(new_vert_array),3 ))

            c=0
            for vert in new_vert_array:
                aux_vert_array[c] = smothing_func(vert , r) 
                c+=1

            #print(aux_vert_array)
            
        elif not smooth:
        
            aux_vert_array = new_vert_array.copy()
            

    
    # Run over ImproveSurfMesh  --> FETK Toolkit
    # Place this program path in constants.py  !
    
    new_face_array , aux_vert_array = Improve_Mesh(new_face_array , aux_vert_array , mesh_info.path , 
                                                  mesh_info.mol_name+ '_' + str(dens) + output_suffix )
    
    
    vert_and_face_arrays_to_text_and_mesh( name , aux_vert_array ,
                                                new_face_array.astype(int)[:] , output_suffix, dens , Self_build=True)

    grid = Grid_loader( name , dens , output_suffix )
    #print('New mesh:')
    #grid.plot()

    N_elements = len(face_array)
        
    return S_trad , S_Cooper , S_Zeb , N_elements , fin_time


def rutine(name , N_it, percentajes , r = 4.):
    
    #name = 'sphere_excent'
    
    if name == 'sphere_cent':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,  1.E-12 ]]  )
        q = np.array( [1.] )
    
    if name == 'sphere_excent':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,   0.5 ]]  )
        q = np.array( [1.] )
    
    if name == 'sphere_dip_C5':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,   1.E-12 ] ,
                         [  1.E-12 ,  1.E-12 ,   0.5    ] ])
        q = np.array( [1. , 1. ] )
    
    if name == 'sphere_dip': 
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,    0.5 ] ,
                         [  1.E-12 ,  1.E-12 ,   -0.5    ] ])
        q = np.array( [1. , 1. ] )
        
    if name == 'sphere_dip_12':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,    0.5    ] ,
                         [  1.E-12 ,  1.E-12 ,   -0.5    ] ])
        q = np.array( [1. , 2. ] )
        
    #for percentaje in percentajes:
    if True:
        
        percentaje = 0.1
        #percentaje = 1.5
        
        #for s in (True,False):
        if True:
            
            s = True
            suffixes = suffix_names(N_it)

            #x_q = np.array( [[  1.E-12 ,  1.E-12 ,   1.E-12 ]]  ) #,
                           #  [  1.E-12 ,  1.E-12 ,  -0.5    ] ])
            #q = np.array( [1.] ) #, 2. ]) #, 1. ] )

            #r = 1.
            #name = 'sphere_cent'
            dens = 0

            path = os.path.join( 'Molecule' , name , str( int(percentaje*100)) , str(s) )            
            os.system('mkdir -p {0}'.format( path ) )

            #os.system('chmod -R a+xwr {0}/'.format('.'))

            bempp.api.set_ipython_notebook_viewer()
            bempp.api.PLOT_BACKEND = "ipython_notebook"

            # Creates Initial Decahedron
            dec_v_array , dec_f_array = decahedron_points(r)
            vert_and_face_arrays_to_text_and_mesh( name, dec_v_array , dec_f_array+1 , '-s0' 
                                                      , dens=0 , Self_build=True)
            # Refines to obtain a better aproach of a sphere
            main(name , dens , '-s0' , '-s1' , percentaje=2.5 , r=r , x_q=x_q , q=q , smooth = True)


            #grid = Grid_loader( name , 0 , '-s0' )
            #grid.plot()

            Resultados = open('Molecule/{0}/Resultados_{0}.txt'.format( name ) , 'w+')

            Resultados.write('x_q = \n')
            Resultados.write(str(x_q) )
            Resultados.write('\n q =   \n')
            Resultados.write(str(  q) )
            Resultados.write('  \n')


            Resultados.write('Percentaje/100 & Number of elements & Strad & SCoper & SZeb & smooth & time \n')
            Resultados.close()

            c=1
            for i in suffixes[1:-1]:

                line = main(name , dens , suffixes[c]  , suffixes[c+1] , percentaje , r , x_q , q , smooth = s )
                
                
                print(c , suffixes[c] , suffixes[c+1] )
                
                Resultados = open('Molecule/{0}/Resultados_{0}.txt'.format( name ) , 'a')
                Resultados.write('{4:.2f} & {3:d} & {0:.10f} & {1:.10f} & {2:.10f} & {5} & {6:.10f}\n'.format(
                            line[0] , line[1], line[2] , line[3] , percentaje, str(s) , line[4]))
                Resultados.close()

                #res = np.vstack( (res , np.array( (name , line[0] , line[1], line[2]
                #                                  , line[3] , percentaje, str(s) , line[4])) ) )
                c+=1
                
                #os.system( "python3.6 File_converter_python3.py" )

                if c==4 and percentaje == 1.5: break

            #R   = 1.

            a   = 1.
            N   = 20

            f_ex = solution(q, x_q, ep_m, ep_s, r, k , a, N)
            print('Exact Solution: {0:5f}--------------------------------------------'.format(f_ex))
            #print(solution_2(q, x_q, ep_m, ep_s, r, kappa, a, N))
            Resultados = open('Molecule/{0}/Resultados_{0}.txt'.format( name ) , 'a')
            Resultados.write('Exact solution: {0:10f}'.format(f_ex))
            Resultados.close()

            moving_files(name , percentaje , s)
            
    return None


#input_suffix = '-s0'
#output_suffix = '-s1'

#res = np.empty(  (0,8) )
percentajes = ( 0.1 , 1.5 )
N_it = 4

name = 'sphere_cent'
rutine(name , N_it, percentajes )

name = 'sphere_excent'
rutine(name , N_it, percentajes , r = 1.)

In [ ]:
os.system( './usr/bin/python3.6 /home/vicente/Documentos/BEM/File_converter_python3.py' )

In [ ]:
os.system("python3.6 File_converter_python3.py" )

In [ ]:
os.system('python3 File_converter_python3.py' )

In [ ]:
suffix_names(5)

In [ ]:
suffixes[1:-1] , suffixes

In [ ]:
juan = np.array(( [ 1. , 2. , 3. ] ,
                  [ 2. , 1E-4 , 2]))

str(juan)


In [ ]:

vert_array , face_array = decahedron_points(1.)
    

from mpl_toolkits import mplot3d



%matplotlib inline

import matplotlib.pyplot as plt

x = vert_array[:,0]
y = vert_array[:,1]
z = vert_array[:,2]
col = np.ones((11,3))

ax.set_xlim( -10. , 10. )
ax.set_ylim( -10. , 10. )
ax.set_zlim( -10. , 10. )

ax = plt.axes(projection='3d')
ax.scatter(x, y, z, c=z);




In [ ]:
new_vert = (vert_array[6] + vert_array[10])*.5
smothing_func(new_vert , 0.5) , new_vert

In [ ]:
from Grid_Maker_R2  import *
import bempp.api

#vert_array , face_array = decahedron_points(.5)
bempp.api.set_ipython_notebook_viewer()
bempp.api.PLOT_BACKEND = "ipython_notebook"

#vert_and_face_arrays_to_text_and_mesh('sphere' , vert_array , face_array+1 , '-s' 
#                                          , dens=0 , Self_build=True)

grid = Grid_loader('sphere_cent' , 0 , '-s0' )

#grid.plot()

a= 0
def test(x, n, domain_index, result):
    
    global a 
    a+=1
    #print(a)
    result[:] = a
    
dirichl_space = bempp.api.function_space(grid,  "P", 2)
    
testtt = bempp.api.GridFunction(dirichl_space, fun=test)
len(testtt.coefficients)

In [ ]:
from bempp.api.integration import gauss_triangle_points_and_weights

face_txt = open('Molecule/sphere_cent/10/True/sphere_cent_0-s0.face','r').read().split('\n')
vert_txt = open('Molecule/sphere_cent/10/True/sphere_cent_0-s0.vert','r').read().split('\n')

face_array = np.empty((0,3))
vert_array = np.empty((0,3))

for line in face_txt[:-1]:
    face_array = np.vstack((face_array , line.split()))
for line in vert_txt[:-1]:
    vert_array = np.vstack((vert_array , line.split()))
face_array = face_array.astype(int)
vert_array = vert_array.astype(float)

face_1 = face_array[0]

juan = grid.leaf_view.entity_iterator(0)


accuracy_order = 20
points, weights = gauss_triangle_points_and_weights(accuracy_order)
    
dirichl_space.evaluate_local_basis( juan , points , [1,2,3]  )

In [ ]:
dirichl_space.evaluate_local_basis?

In [ ]:
import bempp.api, numpy as np
from math import pi
import os
import time

from constants import values
from constants import mesh_info

from bempp.api.operators.potential import laplace as lp
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

from Grid_Maker_R2  import *
from Mesh_Ref_V2    import *
from quadrature     import *
from Potential_Solver import *
#from bem_parameters import *

x_q = np.array( ([0. , 0. , 0. ] , [0. , 0.5 , 0.] ))
q = np.array( (1. , 2.) )

r = 0.5

global mol_name , mesh_density , suffix , path , q , x_q , phi_space , phi_order , u_space , u_order

#def main(name , dens , input_suffix , output_suffix , percentaje , smooth = False , refine = True):
    
init_time = time.time()

mesh_info.mol_name     = name
mesh_info.mesh_density = dens
mesh_info.suffix       = input_suffix
mesh_info.path         = os.path.join('Molecule' , mesh_info.mol_name)

mesh_info.q , mesh_info.x_q = q , x_q

mesh_info.u_space , mesh_info.u_order     = 'DP' , 0
mesh_info.phi_space , mesh_info.phi_order = 'P' , 1
mesh_info.u_s_space , mesh_info.u_s_order = 'P' , 1


bempp.api.set_ipython_notebook_viewer()
bempp.api.PLOT_BACKEND = "ipython_notebook"

grid = Grid_loader( mesh_info.mol_name , mesh_info.mesh_density , mesh_info.suffix )

dirichl_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)
neumann_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order) 
dual_to_dir_s_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)

u_s  = bempp.api.GridFunction(dirichl_space_u, fun=u_s_G )
du_s = bempp.api.GridFunction(neumann_space_u, fun=du_s_G)

u_h , du_h = harmonic_component(dirichl_space_u , neumann_space_u , dual_to_dir_s_u , u_s , du_s)
u_r , du_r = regular_component(dirichl_space_u , neumann_space_u , dual_to_dir_s_u , du_s , du_h)

S_trad = S_trad_calc( dirichl_space_u, neumann_space_u , u_h , du_h , u_r , du_r)
    
endt_time = time.time()
fin_time = endt_time - init_time

dirichl_space_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order)
neumann_space_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order) 
dual_to_dir_s_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order)

phi , dphi = adjoint_equation( dirichl_space_phi , neumann_space_phi , dual_to_dir_s_phi)

aux_path = '_'+str(mesh_info.mesh_density)+ mesh_info.suffix

face_array = text_to_list(mesh_info.mol_name , aux_path , '.face' , info_type=int  )
vert_array = text_to_list(mesh_info.mol_name , aux_path , '.vert' , info_type=float)
    
S_Cooper , S_Cooper_i = S_Cooper_calc( face_array , vert_array , phi , dphi , u_r+u_h , du_r+du_h , 25)
    
dirichl_space_u_s  = bempp.api.function_space(grid,  mesh_info.u_s_space , mesh_info.u_s_order )
neumann_space_du_s = bempp.api.function_space(grid,  mesh_info.u_s_space , mesh_info.u_s_order )

u_s  = bempp.api.GridFunction(dirichl_space_u_s , fun=u_s_G )
du_s = bempp.api.GridFunction(neumann_space_du_s, fun=du_s_G)      
    
S_Zeb    , S_Zeb_i    = S_Zeb_calc( face_array , vert_array , phi , dphi , u_s , du_s , 25)
                            # Zeb_aproach_with_u_s_Teo( face_array , vert_array , phi , dphi , 25)
        
const_space = bempp.api.function_space(grid,  "DP", 0)

Solv_i_Cooper_Func = bempp.api.GridFunction(const_space, fun=None, coefficients=S_Cooper_i[:,0])
Solv_i_Zeb_Func    = bempp.api.GridFunction(const_space, fun=None, coefficients=S_Zeb_i[:,0])

dif =S_Cooper_i-S_Zeb_i

dif_F = bempp.api.GridFunction(const_space, fun=None, coefficients=np.abs(dif[:,0] ) )
#dif_F.plot()    
    
bempp.api.export('Molecule/' + name +'/' + name + '_{0}{1}.vtk'.format( dens,input_suffix )
                     , grid_function = dif_F , data_type = 'element')
    
#if False:
#    status = value_assignor_starter(face_array , np.abs(dif[:,0]) , percentaje) #final_status(face_array , dif[:,0] , 2 )
#    const_space = bempp.api.function_space(grid,  "DP", 0)
#    Status    = bempp.api.GridFunction(const_space, fun=None, coefficients=status)
#    Status.plot()

if refine:
    new_face_array , new_vert_array = mesh_refiner(face_array , vert_array , np.abs(dif[:,0]) , percentaje )

    if smooth:
        
        aux_vert_array = np.zeros(( len(new_vert_array),3 ))
        for vert in new_vert_array:
            aux_vert_array = smothing_func(vert , r) 

    vert_and_face_arrays_to_text_and_mesh( name , new_vert_array ,
                                            new_face_array.astype(int)[:] , output_suffix, dens , Self_build=True)

    grid = Grid_loader( name , dens , output_suffix )
    print('New mesh:')
    grid.plot()
    
    N_elements = len(face_array)
    
#return S_trad , S_Cooper , S_Zeb , N_elements , fin_time
   #return dif[:,0]
    
print('Test')

In [ ]:
q

In [12]:
%%writefile main_sphere.py

# August 13th 2019
# This rutine uses the three term splitting formulation

# Fixed mayor bugs!!!!!!

import bempp.api, numpy as np
from math import pi
import os
import time

from constants import values
from constants import mesh_info

from bempp.api.operators.potential import laplace as lp
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

from Grid_Maker_R2  import *
from Mesh_Ref_V2    import *
from quadrature     import *
from Potential_Solver import *
from sphere_geom    import *
#from bem_parameters import *
from File_converter_python2 import *

from analytical     import *


#global mol_name , mesh_density , suffix , path , q , x_q , phi_space , phi_order , u_space , u_order

def main(name , dens , input_suffix , output_suffix , percentaje , r , x_q
         , q , smooth = False , refine = True  ):
    
    init_time = time.time()

    mesh_info.mol_name     = name
    mesh_info.mesh_density = dens
    mesh_info.suffix       = input_suffix
    mesh_info.path         = os.path.join('Molecule' , mesh_info.mol_name)
    
    print('{1:.0f} {0} '.format(mesh_info.suffix , percentaje * 100.) )

    mesh_info.q , mesh_info.x_q = q , x_q

    mesh_info.u_space , mesh_info.u_order     = 'DP' , 0
    mesh_info.phi_space , mesh_info.phi_order =  'P' , 1
    mesh_info.u_s_space , mesh_info.u_s_order =  'P' , 1


    bempp.api.set_ipython_notebook_viewer()
    bempp.api.PLOT_BACKEND = "ipython_notebook"

    grid = Grid_loader( mesh_info.mol_name , mesh_info.mesh_density , mesh_info.suffix )
        
    dirichl_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)
    neumann_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order) 
    dual_to_dir_s_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)

    U, dU , U_R , dU_R = U_and_U_Reac(dirichl_space_u , neumann_space_u , dual_to_dir_s_u )
    
    #(U_R  - ( u_r +  u_h)).plot()
    #(dU_R - (du_r + du_h)).plot()
    
    
    S_trad = S_trad_calc_R( dirichl_space_u, neumann_space_u , U , dU )
    
    endt_time = time.time()
    fin_time = endt_time - init_time

    dirichl_space_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order)
    neumann_space_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order) 
    dual_to_dir_s_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order)

    phi , dphi = adjoint_equation( dirichl_space_phi , neumann_space_phi , dual_to_dir_s_phi)

    aux_path = '_'+str(mesh_info.mesh_density)+ mesh_info.suffix

    face_array = np.transpose(grid.leaf_view.elements)+1
    vert_array = np.transpose(grid.leaf_view.vertices)

    S_Cooper , S_Cooper_i = S_Cooper_calc( face_array , vert_array , phi , dphi , U_R , dU_R , 25)

    dirichl_space_u_s  = bempp.api.function_space(grid,  mesh_info.u_s_space , mesh_info.u_s_order )
    neumann_space_du_s = bempp.api.function_space(grid,  mesh_info.u_s_space , mesh_info.u_s_order )

    u_s  = bempp.api.GridFunction(dirichl_space_u_s , fun=u_s_G )
    du_s = bempp.api.GridFunction(neumann_space_du_s, fun=du_s_G)      

    S_Zeb    , S_Zeb_i    =Zeb_aproach_with_u_s_Teo( face_array , vert_array , phi , dphi , 25)
                                 # S_Zeb_calc( face_array , vert_array , phi , dphi , u_s , du_s , 25)

    const_space = bempp.api.function_space(grid,  "DP", 0)

    Solv_i_Cooper_Func = bempp.api.GridFunction(const_space, fun=None, coefficients=S_Cooper_i[:,0])
    Solv_i_Zeb_Func    = bempp.api.GridFunction(const_space, fun=None, coefficients=S_Zeb_i[:,0])

    dif =S_Cooper_i-S_Zeb_i

    dif_F = bempp.api.GridFunction(const_space, fun=None, coefficients=np.abs(dif[:,0] ) )
    #dif_F.plot()    

    bempp.api.export('Molecule/' + name +'/' + name + '_{0}{1}.vtk'.format( dens,input_suffix )
                         , grid_function = dif_F , data_type = 'element')

    #if False:
    #    status = value_assignor_starter(face_array , np.abs(dif[:,0]) , percentaje) #final_status(face_array , dif[:,0] , 2 )
    #    const_space = bempp.api.function_space(grid,  "DP", 0)
    #    Status    = bempp.api.GridFunction(const_space, fun=None, coefficients=status)
    #    Status.plot()

    if refine:

        new_face_array , new_vert_array = mesh_refiner(face_array , vert_array , np.abs(dif[:,0]) , percentaje )

        if smooth:

            aux_vert_array = np.zeros(( len(new_vert_array),3 ))

            c=0
            for vert in new_vert_array:
                aux_vert_array[c] = smothing_func(vert , r) 
                c+=1

            #print(aux_vert_array)
            
        elif not smooth:
        
            aux_vert_array = new_vert_array.copy()
            

    
    # Run over ImproveSurfMesh  --> FETK Toolkit
    # Place this program path in constants.py  !
    
    new_face_array , aux_vert_array = Improve_Mesh(new_face_array , aux_vert_array , mesh_info.path , 
                                                  mesh_info.mol_name+ '_' + str(dens) + output_suffix )
    
    vert_and_face_arrays_to_text_and_mesh( name , aux_vert_array ,
                                                new_face_array.astype(int)[:] , output_suffix, dens , Self_build=True)

    grid = Grid_loader( name , dens , output_suffix )
    
    #grid.plot()

    N_elements = len(face_array)
    
    print(N_elements)
        
    return S_trad , S_Cooper , S_Zeb , N_elements , fin_time


def rutine(name , N_it, percentajes , r = 1.):
    
    if name == 'sphere_cent':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,  1.E-12 ]]  )
        q = np.array( [1.] )
    
    if name == 'sphere_excent':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,   r/2. ]]  )
        q = np.array( [1.] )
    
    if name == 'sphere_dip_C5':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,   1.E-12 ] ,
                         [  1.E-12 ,  1.E-12 ,   0.5    ] ])
        q = np.array( [1. , 1. ] )
    
    if name == 'sphere_dip': 
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,    0.5 ] ,
                         [  1.E-12 ,  1.E-12 ,   -0.5    ] ])
        q = np.array( [1. , 1. ] )
        
    if name == 'sphere_dip_12':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,    0.5    ] ,
                         [  1.E-12 ,  1.E-12 ,   -0.5    ] ])
        q = np.array( [1. , 2. ] )
        
    #for percentaje in percentajes:
    if True:
        
        percentaje = percentajes
        
        #for s in (True,False):
        if True:
            
            s = True
            suffixes = suffix_names(N_it)

            #x_q = np.array( [[  1.E-12 ,  1.E-12 ,   1.E-12 ]]  ) #,
                           #  [  1.E-12 ,  1.E-12 ,  -0.5    ] ])
            #q = np.array( [1.] ) #, 2. ]) #, 1. ] )

            #r = 1.
            #name = 'sphere_cent'
            dens = 0

            path = os.path.join( 'Molecule' , name , str( int(percentaje*100)) , str(s) )            
            os.system('mkdir -p {0}'.format( path ) )

            #os.system('chmod -R a+xwr {0}/'.format('.'))

            bempp.api.set_ipython_notebook_viewer()
            bempp.api.PLOT_BACKEND = "ipython_notebook"

            # Creates Initial Decahedron
            dec_v_array , dec_f_array = decahedron_points(r)
            vert_and_face_arrays_to_text_and_mesh( name, dec_v_array , dec_f_array+1 , '-s0' 
                                                      , dens=0 , Self_build=True)
            # Refines to obtain a better aproach of a sphere
            main(name , dens , '-s0' , '-s1' , percentaje=2.5 , r=r , x_q=x_q , q=q , smooth = True)
            
            #main(name , dens , '-s1' , '-s2' , percentaje=2.5 , r=r , x_q=x_q , q=q , smooth = True)


            #grid = Grid_loader( name , 0 , '-s0' )
            #grid.plot()

            Resultados = open('Molecule/{0}/Resultados_{0}.txt'.format( name ) , 'w+')

            Resultados.write('x_q = \n')
            Resultados.write(str(x_q) )
            Resultados.write('\n q =   \n')
            Resultados.write(str(  q) )
            Resultados.write('  \n')


            Resultados.write('Percentaje & Number of elements & Strad & SAprox & SEx & smooth & time \n')
            Resultados.close()

            c=1
            for i in suffixes[1:-1]:

                line = main(name , dens , suffixes[c]  , suffixes[c+1] , percentaje , r , x_q , q , smooth = s )
                
                
                print(c , suffixes[c] , suffixes[c+1] )
                
                Resultados = open('Molecule/{0}/Resultados_{0}.txt'.format( name ) , 'a')
                Resultados.write('{4:.2f} & {3:d} & {0:.10f} & {1:.10f} & {2:.10f} & {5} & {6:.10f}\n'.format(
                            line[0] , line[1], line[2] , line[3] , percentaje, str(s) , line[4]))
                Resultados.close()

                #res = np.vstack( (res , np.array( (name , line[0] , line[1], line[2]
                #                                  , line[3] , percentaje, str(s) , line[4])) ) )
                c+=1
                
                #os.system( "python3.6 File_converter_python3.py" )

                if c==5 and percentaje == 1.5: break

            #R   = 1.

            a   = 1.
            N   = 20

            f_ex = solution(q, x_q, ep_m, ep_s, r, k , a, N)
            print('Exact Solution: {0:5f}--------------------------------------------'.format(f_ex))
            #print(solution_2(q, x_q, ep_m, ep_s, r, kappa, a, N))
            Resultados = open('Molecule/{0}/Resultados_{0}.txt'.format( name ) , 'a')
            Resultados.write('Exact solution: {0:10f}'.format(f_ex))
            Resultados.close()

            moving_files(name , percentaje , s)
            
    return None

N_it = 31
percentajes = 0.1

name = 'sphere_cent'
rutine(name , N_it, percentajes , r = 1.)
rutine(name , 5   , 1.5         , r = 1.)

name = 'sphere_excent'
rutine(name , N_it, percentajes , r = 1.)
rutine(name , 5   , 1.5         , r = 1.)

Overwriting main_sphere.py


In [30]:
%%writefile main_sphere_Adj.py

# October 10 - 2019
# This rutine uses the three term splitting formulation

# Same as the rutine defined above, uses an uniform refined mesh for the Adjoint

import bempp.api, numpy as np
from math import pi
import os
import time

from constants import values
from constants import mesh_info

from bempp.api.operators.potential import laplace as lp
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

from Grid_Maker_R2  import *
from Mesh_Ref_V2    import *
from quadrature     import *
from Potential_Solver import *
from sphere_geom    import *
#from bem_parameters import *
from File_converter_python2 import *

from analytical     import *


#global mol_name , mesh_density , suffix , path , q , x_q , phi_space , phi_order , u_space , u_order

def main(name , dens , input_suffix , output_suffix , percentaje , r , x_q
         , q , smooth = False , refine = True  ):
    
    init_time = time.time()

    mesh_info.mol_name     = name
    mesh_info.mesh_density = dens
    mesh_info.suffix       = input_suffix
    mesh_info.path         = os.path.join('Molecule' , mesh_info.mol_name)
    
    print('{1:.0f} {0} '.format(mesh_info.suffix , percentaje * 100.) )

    mesh_info.q , mesh_info.x_q = q , x_q

    mesh_info.u_space , mesh_info.u_order     = 'DP' , 0
    mesh_info.phi_space , mesh_info.phi_order =  'P' , 1
    mesh_info.u_s_space , mesh_info.u_s_order =  'P' , 1


    bempp.api.set_ipython_notebook_viewer()
    bempp.api.PLOT_BACKEND = "ipython_notebook"

    grid = Grid_loader( mesh_info.mol_name , mesh_info.mesh_density , mesh_info.suffix )
        
    dirichl_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)
    neumann_space_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order) 
    dual_to_dir_s_u = bempp.api.function_space(grid,  mesh_info.u_space, mesh_info.u_order)
    
    space_time = time.time() - init_time

    U, dU , operators_time , matrix_assembly_time , solv_time , U_it_count= U_tot(dirichl_space_u  , 
                                                                  neumann_space_u , dual_to_dir_s_u )
    
    S_trad = S_trad_calc_R( dirichl_space_u, neumann_space_u , U , dU )
    
    endt_time = time.time()
    fin_time = endt_time - init_time
    
    U_R , dU_R = U_Reac(U, dU , dirichl_space_u , neumann_space_u )
    
    ref_init_time = time.time()

    dirichl_space_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order)
    neumann_space_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order) 
    dual_to_dir_s_phi = bempp.api.function_space(grid,  mesh_info.phi_space , mesh_info.phi_order)
    
    phi , dphi , _ = adjoint_equation( dirichl_space_phi , neumann_space_phi , dual_to_dir_s_phi)

    aux_path = '_'+str(mesh_info.mesh_density)+ mesh_info.suffix

    face_array = np.transpose(grid.leaf_view.elements)+1
    vert_array = np.transpose(grid.leaf_view.vertices)

    S_Cooper , S_Cooper_i = S_Cooper_calc( face_array , vert_array , phi , dphi , U_R , dU_R , 25)

    dirichl_space_u_s  = bempp.api.function_space(grid,  mesh_info.u_s_space , mesh_info.u_s_order )
    neumann_space_du_s = bempp.api.function_space(grid,  mesh_info.u_s_space , mesh_info.u_s_order )

    u_s  = bempp.api.GridFunction(dirichl_space_u_s , fun=u_s_G )
    du_s = bempp.api.GridFunction(neumann_space_du_s, fun=du_s_G)      

    S_Zeb    , S_Zeb_i  , it_adj_count  = S_Zeb_in_Adjoint_Mesh(mesh_info.mol_name , face_array , vert_array , dens , input_suffix , 25)

    const_space = bempp.api.function_space(grid,  "DP", 0)

    Solv_i_Cooper_Func = bempp.api.GridFunction(const_space, fun=None, coefficients=S_Cooper_i[:,0])
    Solv_i_Zeb_Func    = bempp.api.GridFunction(const_space, fun=None, coefficients=S_Zeb_i[:,0])

    dif =S_Cooper_i-S_Zeb_i

    dif_F = bempp.api.GridFunction(const_space, fun=None, coefficients=np.abs(dif[:,0] ) )
    #dif_F.plot()    

    bempp.api.export('Molecule/' + name +'/' + name + '_{0}{1}.vtk'.format( dens,input_suffix )
                         , grid_function = dif_F , data_type = 'element')

    #if False:
    #    status = value_assignor_starter(face_array , np.abs(dif[:,0]) , percentaje) #final_status(face_array , dif[:,0] , 2 )
    #    const_space = bempp.api.function_space(grid,  "DP", 0)
    #    Status    = bempp.api.GridFunction(const_space, fun=None, coefficients=status)
    #    Status.plot()

    if refine:

        new_face_array , new_vert_array = mesh_refiner(face_array , vert_array , np.abs(dif[:,0]) , percentaje )

        if smooth:

            aux_vert_array = np.zeros(( len(new_vert_array),3 ))

            c=0
            for vert in new_vert_array:
                aux_vert_array[c] = smothing_func(vert , r) 
                c+=1

            #print(aux_vert_array)
            
        elif not smooth:
        
            aux_vert_array = new_vert_array.copy()
            

    
    # Run over ImproveSurfMesh  --> FETK Toolkit
    # Place this program path in constants.py  !
    
    new_face_array , aux_vert_array = Improve_Mesh(new_face_array , aux_vert_array , mesh_info.path , 
                                                  mesh_info.mol_name+ '_' + str(dens) + output_suffix )
    
    vert_and_face_arrays_to_text_and_mesh( name , aux_vert_array ,
                                                new_face_array.astype(int)[:] , output_suffix, dens , Self_build=True)

    grid = Grid_loader( name , dens , output_suffix )
    
    #grid.plot()

    N_elements = len(face_array)
    
    print(N_elements)
    
    ref_end_time = time.time()
    ref_fin_time = ref_end_time - ref_init_time
    
        
    return (S_trad , S_Cooper , S_Zeb , N_elements , fin_time , ref_fin_time , operators_time ,
            matrix_assembly_time , solv_time , it_adj_count , U_it_count)

def rutine(name , N_it, percentajes , r = 1.):
    
    if name == 'sphere_cent':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,  1.E-12 ]]  )
        q = np.array( [1.] )
    
    if name == 'sphere_excent':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,   r/2. ]]  )
        q = np.array( [1.] )
    
    if name == 'sphere_dip_C5':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,   1.E-12 ] ,
                         [  1.E-12 ,  1.E-12 ,   0.5    ] ])
        q = np.array( [1. , 1. ] )
    
    if name == 'sphere_dip': 
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,    0.5 ] ,
                         [  1.E-12 ,  1.E-12 ,   -0.5    ] ])
        q = np.array( [1. , 1. ] )
        
    if name == 'sphere_dip_12':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,    0.5    ] ,
                         [  1.E-12 ,  1.E-12 ,   -0.5    ] ])
        q = np.array( [1. , 2. ] )
        
    if name == 'special':
        x_q = np.array( [[  1.E-12 ,  1.E-12 ,  0.62 ],
                             [  1.E-12 ,  0.62*np.cos(np.pi*1.5 + 5.*np.pi/180.) ,
                                                      0.62*np.sin(np.pi*1.5 + 5.*np.pi/180. ) ] ,
                             [  1.E-12 ,  0.62*np.cos(np.pi*1.5 - 5.*np.pi/180.) ,
                                                      0.62*np.sin(np.pi*1.5 - 5.*np.pi/180. )  ]
                            ])
        q = np.array( [1. , 1. , -1.]) 
    #for percentaje in percentajes:
    if True:
        
        percentaje = percentajes
        
        #for s in (True,False):
        if True:
            
            s = True
            suffixes = suffix_names(N_it)

            #x_q = np.array( [[  1.E-12 ,  1.E-12 ,   1.E-12 ]]  ) #,
                           #  [  1.E-12 ,  1.E-12 ,  -0.5    ] ])
            #q = np.array( [1.] ) #, 2. ]) #, 1. ] )

            #r = 1.
            #name = 'sphere_cent'
            dens = 0

            path = os.path.join( 'Molecule' , name , str( int(percentaje*100)) , str(s) )            
            os.system('mkdir -p {0}'.format( path ) )

            bempp.api.set_ipython_notebook_viewer()
            bempp.api.PLOT_BACKEND = "ipython_notebook"

            # Creates Initial Decahedron
            dec_v_array , dec_f_array = decahedron_points(r)
            vert_and_face_arrays_to_text_and_mesh( name, dec_v_array , dec_f_array+1 , '-s0' 
                                                      , dens=0 , Self_build=True)
            # Refines to obtain a better aproach of a sphere
            main(name , dens , '-s0' , '-s1' , percentaje=2.5 , r=r , x_q=x_q , q=q , smooth = True)

            Resultados = open('Molecule/{0}/Resultados_{0}.txt'.format( name ) , 'w+')

            Resultados.write('x_q = \n')
            Resultados.write(str(x_q) )
            Resultados.write('\n q =   \n')
            Resultados.write(str(  q) )
            Resultados.write('  \n')


            Resultados.write('Percentaje & Number of elements & Strad & SAprox & SEx & smooth & time \n')
            Resultados.close()

            c=1
            for i in suffixes[1:-1]:

                (S_trad , S_Cooper , S_Zeb , N_elements , fin_time , ref_time ,
                 operators_time , matrix_assembly_time , solv_time ,
                 it_adj_count , U_it_count) = main(name , dens , suffixes[c]  , suffixes[c+1] , percentaje , r , x_q , q , smooth = s )
                
                
                print(c , suffixes[c] , suffixes[c+1] )
                
                Resultados = open('Molecule/{0}/Resultados_{0}.txt'.format( name ) , 'a')
                
                text = '- & '+str(i) + ' & {0:d} & {1:.10f} & {2:.10f} & {3:.10f} & {4:.10f} & {5:.10f}'.format( 
                    N_elements , S_trad , S_Cooper , S_Zeb , fin_time, ref_time  
                ) +' & {0:.10f} & {1:.10f} & {2:.10f} & {3:d} & {4:d} \n'.format(  
                        operators_time , matrix_assembly_time , solv_time , it_adj_count , U_it_count  )
                Resultados.write(text)
                #Resultados.write('{4:.2f} & {3:d} & {0:.10f} & {1:.10f} & {2:.10f} & {5} & {6:.10f}\n'.format(
                #            line[0] , line[1], line[2] , line[3] , percentaje, str(s) , line[4]))
                Resultados.close()

                #res = np.vstack( (res , np.array( (name , line[0] , line[1], line[2]
                #                                  , line[3] , percentaje, str(s) , line[4])) ) )
                c+=1
                
                #os.system( "python3.6 File_converter_python3.py" )

                if c==5 and percentaje == 1.5: break

            #R   = 1.

            a   = 1.
            N   = 20

            f_ex = solution(q, x_q, ep_m, ep_s, r, k , a, N)
            print('Exact Solution: {0:5f}--------------------------------------------'.format(f_ex))
            #print(solution_2(q, x_q, ep_m, ep_s, r, kappa, a, N))
            Resultados = open('Molecule/{0}/Resultados_{0}.txt'.format( name ) , 'a')
            Resultados.write('Exact solution: {0:10f}'.format(f_ex))
            
            Resultados.write('Conditions\n')
            
            Resultados.write('Smooth and Use_Gamer: Enabled \n')
            Resultados.write('Adjoint mesh with uniform refinement: Enabled \n')
            Resultados.write('Note: None \n')
            
            Resultados.close()

            moving_files(name , percentaje , s)
            
        
            
    return None

N_it = 31
percentajes = 0.1

name = 'special'
rutine(name , N_it, percentajes , r = 1.)#
rutine(name , 5   , 1.5         , r = 1.)

#name = 'sphere_excent'
#rutine(name , N_it, percentajes , r = 1.)
#rutine(name , 5   , 1.5         , r = 1.)

Writing main_sphere_Adj.py


In [ ]:


# August 19th 2019
# This rutine uses the three term splitting formulation

# TEST RUN FOR BREAKING QUADRATURE.PY!!!!!!!!! 

import bempp.api, numpy as np
from math import pi
import os
import time

from constants import values
from constants import mesh_info

from bempp.api.operators.potential import laplace as lp
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz

from Grid_Maker_R2  import *
from Mesh_Ref_V2    import *
from quadrature     import *
from Potential_Solver import *
from sphere_geom    import *
#from bem_parameters import *
from File_converter_python2 import *

from analytical     import *


#global mol_name , mesh_density , suffix , path , q , x_q , phi_space , phi_order , u_space , u_order

def main_test(name , dens , input_suffix , output_suffix , percentaje , r , x_q
         , q , smooth = False , refine = True  ):
    
    mesh_info.mol_name     = name
    mesh_info.mesh_density = dens
    mesh_info.suffix       = input_suffix
    mesh_info.path         = os.path.join('Molecule' , mesh_info.mol_name)
    
    print('{1:.0f} {0} '.format(mesh_info.suffix , percentaje * 100.) )

    mesh_info.q , mesh_info.x_q = q , x_q
    
    
    
   


    bempp.api.set_ipython_notebook_viewer()
    bempp.api.PLOT_BACKEND = "ipython_notebook"
    
    # Esto carga cualquier malla
    grid = Grid_loader( mesh_info.mol_name , mesh_info.mesh_density , mesh_info.suffix )
    
    A_space , A_order = 'DP' , 1
    B_space , B_order = 'DP' , 0

    dirichl_space_A = bempp.api.function_space(grid,  A_space , A_order)
    neumann_space_A = bempp.api.function_space(grid,  A_space , A_order) 
    dual_to_dir_s_A = bempp.api.function_space(grid,  A_space , A_order)

    # Función sobre A
    U_A = bempp.api.GridFunction(dirichl_space_A , fun =  z_value)
    
    U_A.plot()
    
    aux_path = '_'+str(mesh_info.mesh_density)+ mesh_info.suffix
    
    face_array = text_to_list(mesh_info.mol_name , aux_path , '.face' , info_type=int  )
    vert_array = text_to_list(mesh_info.mol_name , aux_path , '.vert' , info_type=float)
        
    return grid , U_A , face_array , vert_array


N_it = 5
percentajes = ''

#name = 'sphere_cent'
#rutine(name , N_it, percentajes , r = 1.)

name = 'sphere_excent'
mesh , U , face_array , vert_array = main_test(name , 0 , '-s0' , '-s1' , 0.1 , 1. , '' , '' )

In [ ]:
np.shape(U.coefficients ) , np.shape(mesh.leaf_view.elements)[1]*3

In [ ]:
U.coefficients

In [ ]:
sorted_vert = np.empty((0,1))

vert_array = np.transpose(mesh.leaf_view.vertices)

for face in np.transpose(mesh.leaf_view.elements):
    for i in face:
        sorted_vert = np.vstack((vert_array[i,0],sorted_vert))
sorted_vert

In [ ]:
U.coefficients.reshape(20 , 3) #- mesh.leaf_view.vertices

In [ ]:
for i in mesh.leaf_view.elements[0]:
    print(mesh.leaf_view.vertices[0,i] - U.coefficients[i])

In [ ]:
mesh.leaf_view.vertex_from_index(1)

In [ ]:
np.transpose(mesh.leaf_view.vertices)